# Metody Obliczeniowe w Nauce i Technice
## Laboratorium 12
### Równania różniczkowe i zagadnienie początkowe
#### Mateusz Surjak

## Metoda Rungego-Kutty

Metoda Rungego-Kutty jest to metoda numeryczna do iteracyjnego rozwiązywania równań różniczkowych zwyczajnych. 

In [1]:
from mpmath import *
import numpy as np

def rk4(f, t, x, h, n): 
    t_a = t
    for j in range(1, n+1): 
        k1 = h * f(t, x) 
        k2 = h * f(t + 0.5 * h, x + 0.5 * k1) 
        k3 = h * f(t + 0.5 * h, x + 0.5 * k2) 
        k4 = h * f(t + h, x + k3) 
        x = x + (1.0/6.0)*(k1 + 2 * k2 + 2 * k3 + k4) 
        t = t_a + j*h 
    return x

#### Jakie zalety ma metoda Rungego-Kutty w porównaniu do metody z szeregami Taylora?
Metoda n-tego rzędu z szeregami Tylora wymaga znalezienia wyrażeń dla pochodnej funkcji f względem t do n-tej włącznie. Metoda Rungego-Kutty nie potrzebuje znać wzorów pochodnych.

In [2]:
def fun(t, x):
    return x/t + t*sec(x/t)

def test(t):
    return t*np.arcsin(t)

Za punkt początkowy przyjąłem wartość bliską 0, a nie samo 0 aby zapobiec dzieleniu przez 0.

In [3]:
a = 10**(-30) # 0
# x(a)
x = 0
h = 2**(-7)
t = 1
n = int((t-a)/h)
print(rk4(fun,a,x, h, n) )
print(test(1))

1.51912512047707
1.5707963267948966


Wartość jest bliska wartości rzeczywystej, jeśli zmniejszymy h to wynik jest dokładniejszy.

In [4]:
a = 10**(-30) # 0
# x(a)
x = 0
h = 2**(-12)
t = 1
n = int((t-a)/h)
print(rk4(fun,a,x, h, n) )
print(test(1))

1.5616613388712
1.5707963267948966


Wyniki są bardzo bliskie wynikom prawdziwym.

In [5]:
def fun2(t,x):
    return 100*(np.sin(t) - x)

a = 10**(-30) # 0
# x(a)
x = 0
t = 3
hh = [0.015, 0.02, 0.025, 0.03]
for h in hh:
    n = int((t-a)/h)
    print(f'For h: {h} result ---> {rk4(fun2,a,x, h, n)}' )

For h: 0.015 result ---> 0.15100302946456207
For h: 0.02 result ---> 0.15099613280114313
For h: 0.025 result ---> 0.15094316610112551
For h: 0.03 result ---> 672890582787.5071


Wyniki dla h mniejszego od 0.03 są naprawdę dobre, natomiast dla h = 0.3 wynik jest totalnie nieprawdziwy.

To zjawisko zostało zauważone przez Dahlquista i Bjorcka, jest opisane w książce "The first course in Numerical Analysis". Dokładne rozwiązanie wygląda następująco:
$$y(x) = \frac{sin(x) - .01cos(x) + .01e^{-100x}}{1.0001}$$

Problem leży w tym że składnik $.01e^{-100x}$ wpływa na przedział stabilności w trakcie obliczeń. Dla przypadku z zadania przedział ten wynosi od 0 do 2.78 (liczby zaczerpnięte z w.w książki), co w przeliczeniu na h wynosi (0, 0.0278). To wyjaśnia dlaczego wartość h = 0.03 powoduje niestabilność i wynik jest bardzo rozbieżny z rzeczywistoscią.



## Adaptacyjna metoda Rungego-Kutty-Fehlberga

Metoda adaptacyjna Rungego-Kutty-Fehlberga pozwala zwiększyć dokładność rozwiązania poprzez kontrolowanie błędu i manipulację wartością h.

In [6]:
def rk45( f, t,x,h,E):

    c20  =   2.500000000000000e-01  
    c30  =   3.750000000000000e-01  
    c40  =   9.230769230769231e-01  
    c50  =   1.000000000000000e+00  
    c60  =   5.000000000000000e-01  

    c21 =   2.500000000000000e-01  
    c31 =   9.375000000000000e-02  
    c32 =   2.812500000000000e-01  
    c41 =   8.793809740555303e-01  
    c42 =  -3.277196176604461e+00  
    c43 =   3.320892125625853e+00  
    c51 =   2.032407407407407e+00  
    c52 =  -8.000000000000000e+00  
    c53 =   7.173489278752436e+00  
    c54 =  -2.058966861598441e-01  
    c61 =  -2.962962962962963e-01  
    c62 =   2.000000000000000e+00  
    c63 =  -1.381676413255361e+00  
    c64 =   4.529727095516569e-01  
    c65 =  -2.750000000000000e-01  


    a1  =   1.157407407407407e-01  
    a2  =   0.000000000000000e-00  
    a3  =   5.489278752436647e-01  
    a4  =   5.353313840155945e-01  
    a5  =  -2.000000000000000e-01  

    b1  =   1.185185185185185e-01  
    b2  =   0.000000000000000e-00  
    b3  =   5.189863547758284e-01  
    b4  =   5.061314903420167e-01  
    b5  =  -1.800000000000000e-01  
    b6  =   3.636363636363636e-02  

    k1 = h * f( t,x )
    k2 = h * f(t+c20*h,x+c21*k1 )
    k3 = h * f(t+c30*h,x+c31*k1+c32*k2 )
    k4 = h * f(t+c40*h,x+c41*k1+c42*k2 + c43*k3 )
    k5 = h * f(t+h,x+c51*k1+c52*k2+c53*k3 + c54*k4 )
    k6 = h * f(t+c60*h,x+c61*k1+c62*k2+c63*k3 + c64*k4 + c65*k5 )

    x_4 = x + a1 * k1 + a3 * k3 + a4 * k4 + a5 * k5
    x = x + b1 * k1 + b3 * k3 + b4 * k4 + b5 * k5 + b6 * k6
    t = t+h
    e = abs( x - x_4 )

    return ( x, e ,t)


In [7]:
def RK45_Adaptive(f,t,x,h,tb,itmax,Emax,Emin,hmin,hmax,iflag):
    xsave = 0
    tsave = 0
    sig = 1/2 * (10)**(-5)
    iflag = 1
    k = 0
    d = 0
    E = 0
    while k <= itmax:
        k = k + 1
        if abs(h) < hmin:
            h = np.sign(h)*hmin
        if abs(h) > hmax:
            h = np.sign(h)*hmax
        d = abs(tb-t)
        if d <= abs(h):
            iflag = 0
            if d <= sig * max([abs(tb),abs(t)]):
                break;
            h = np.sign(h)*d
        xsave = x
        tsave = t
        x,E,t = rk45(f,t,x,h,E)
        if iflag == 0:
            break;
        if E < Emin:
            h = 2*h
        if E > Emax:
            h = h/2
            x = xsave
            t = tsave
            k = k - 1
        
    return x


In [8]:
def ff(t,x):
    return 3*x/t+9/2*t-13
# ustawienia do metody adaptacyjnej
itflag  = 1
itmax = 1000
Emax = 10**(-5)
Emin = 10**(-8)
hmin = 10**(-6)
hmax = 1
t = 3
#x(a)
x = 6
h = -0.01
# koniec przedziały - wartość w jakim punkcie nas interesuje
tb = 1/2
print(RK45_Adaptive(ff,t,x,h,tb,itmax,Emax,Emin,hmin,hmax,itflag))
def test_fun(t):
    return t**3 - 9/2*t**2+13/2*t
print(test_fun(1/2))

2.2500031598916395
2.25


H przyjąłem ujemne aby można przyjąć warunek brzegowy jako końcowy punkt przedziału.

Metoda Rungego-Kutty-Fehlberga daje dwa przebliżenia wartości x(t+h), jedno piątego rzędu, drugie czwartego, których różnica jest błędem drugiej wartości. Możemy dzięki temu kontrtolować wartość błędu i sterować wartością h. Pierwsza wartość jest bardziej dokładna dla właściwego h i daje ostateczne przyblizenie. Można sądzić że rzeczywisty błąd jest znacznie mniejszy od błędu przez nas wyliczonego. W skrócie...metoda adaptacyjna pozwala zwiększyć dokładność rozwiązania poprzez kontrolowanie błędu i manipulację wartością h.

Wadą tego rozwiązania jest złożoność czasowa.